Firsly, you need to install the elasticsearch python client. Check our [installation guide]() for more details.
Our notebook will use the same configuration as the docker-compose file. You can change the configuration in 
the notebook if you want to use your own Elasticsearch instance.

In [5]:
import os 
from dotenv import load_dotenv
from pathlib import Path

dotenv_path = Path('.env')
load_dotenv(dotenv_path=dotenv_path)

ES_PORT = os.environ.get("ELASTIC_PORT", None)
print(f"ES_PORT: {ES_PORT}")

ES_PORT: 20542


## Prepapre DB

In [6]:
import pandas as pd 
df = pd.read_csv('demo_db.csv')

In [7]:
# convert df words from string of list to list 
df['words'] = df['words'].apply(lambda x: eval(x))
df.head()

,video_id,frame_id,words
0,C00_V0000,0,"[06:, 29, HTV, :38, HP]"
1,C00_V0000,96,"[H, TV, 29:42, 06:, HD]"
2,C00_V0000,206,"[06:29:46, HD, HTV9]"
3,C00_V0000,378,"[9, GIÁY, 06, :29, HTV, :53, HD, NY, GIÂY, MAI..."
4,C00_V0000,545,"[30, :00, 06, HTV, MP]"


In [9]:
df['index'] = df.apply(lambda x: "{}/{}".format(x['video_id'], x['frame_id']), axis=1)
df.head()

,video_id,frame_id,words,index
0,C00_V0000,0,"[06:, 29, HTV, :38, HP]",C00_V0000/0
1,C00_V0000,96,"[H, TV, 29:42, 06:, HD]",C00_V0000/96
2,C00_V0000,206,"[06:29:46, HD, HTV9]",C00_V0000/206
3,C00_V0000,378,"[9, GIÁY, 06, :29, HTV, :53, HD, NY, GIÂY, MAI...",C00_V0000/378
4,C00_V0000,545,"[30, :00, 06, HTV, MP]",C00_V0000/545


## Create index in Elasticsearch

In [34]:
from elasticsearch.helpers import bulk
from elasticsearch import Elasticsearch, RequestsHttpConnection

es = Elasticsearch(['http://0.0.0.0:9200'], timeout=100, connection_class=RequestsHttpConnection, http_auth=('elastic', '123456'), use_ssl=False, verify_certs=False)

In [36]:
es.indices.create(index='scenetext_db', ignore=400)
es.indices.get_alias("*")

/tmp/ipykernel_180386/3661718185.py:2: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.get_alias("*")


{'scenetext_db': {'aliases': {}}}

In [38]:
def gen_data():
    for i, (vid, fid, words) in enumerate(db_ls):
        yield {
            "_index": 'scenetext_db',
            "_id": f'{vid}/{fid}',
            "_source": {
                'vid': vid,
                'fid': fid,
                'words': words
            }
        }
bulk(es, gen_data())

(224340, [])

In [39]:
# Delete index
# es.indices.delete(index='scenetext_db')